In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

In [4]:
data=pd.read_csv('Fraud_check.csv')
data

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [5]:
data['Undergrad']=np.where(data['Undergrad'].str.contains('YES'),1,0)
data['Urban']=np.where(data['Urban'].str.contains('YES'),1,0)
data

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,0,Single,68833,50047,10,1
1,1,Divorced,33700,134075,18,1
2,0,Married,36925,160205,30,1
3,1,Single,50190,193264,15,1
4,0,Married,81002,27533,28,0
...,...,...,...,...,...,...
595,1,Divorced,76340,39492,7,1
596,1,Divorced,69967,55369,2,1
597,0,Divorced,47334,154058,0,1
598,1,Married,98592,180083,17,0


In [6]:
data1=pd.cut(data['Taxable.Income'],bins=[10003,30000,99619],labels=['Risky','Free'])
data1

0      Free
1      Free
2      Free
3      Free
4      Free
       ... 
595    Free
596    Free
597    Free
598    Free
599    Free
Name: Taxable.Income, Length: 600, dtype: category
Categories (2, object): ['Risky' < 'Free']

In [9]:
data['Tax_Cat']=data1
data

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,Tax_Cat
0,0,Single,68833,50047,10,1,Free
1,1,Divorced,33700,134075,18,1,Free
2,0,Married,36925,160205,30,1,Free
3,1,Single,50190,193264,15,1,Free
4,0,Married,81002,27533,28,0,Free
...,...,...,...,...,...,...,...
595,1,Divorced,76340,39492,7,1,Free
596,1,Divorced,69967,55369,2,1,Free
597,0,Divorced,47334,154058,0,1,Free
598,1,Married,98592,180083,17,0,Free


In [10]:
data['Marital.Status']=data['Marital.Status'].map({'Single':0,'Married':1,'Divorced':2})
data

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,Tax_Cat
0,0,0,68833,50047,10,1,Free
1,1,2,33700,134075,18,1,Free
2,0,1,36925,160205,30,1,Free
3,1,0,50190,193264,15,1,Free
4,0,1,81002,27533,28,0,Free
...,...,...,...,...,...,...,...
595,1,2,76340,39492,7,1,Free
596,1,2,69967,55369,2,1,Free
597,0,2,47334,154058,0,1,Free
598,1,1,98592,180083,17,0,Free


In [12]:
data2=data.drop(['Taxable.Income'],axis=1)
data2

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban,Tax_Cat
0,0,0,50047,10,1,Free
1,1,2,134075,18,1,Free
2,0,1,160205,30,1,Free
3,1,0,193264,15,1,Free
4,0,1,27533,28,0,Free
...,...,...,...,...,...,...
595,1,2,39492,7,1,Free
596,1,2,55369,2,1,Free
597,0,2,154058,0,1,Free
598,1,1,180083,17,0,Free


In [13]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    int64 
 1   Marital.Status   600 non-null    int64 
 2   City.Population  600 non-null    int64 
 3   Work.Experience  600 non-null    int64 
 4   Urban            600 non-null    int64 
 5   Tax_Cat          599 non-null    object
dtypes: int64(5), object(1)
memory usage: 28.2+ KB


In [14]:
data2['Tax_Cat']=np.where(data1['Tax_Cat'].str.contains('Free'),0,1)
data2

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban,Tax_Cat
0,0,0,50047,10,1,0
1,1,2,134075,18,1,0
2,0,1,160205,30,1,0
3,1,0,193264,15,1,0
4,0,1,27533,28,0,0
...,...,...,...,...,...,...
595,1,2,39492,7,1,0
596,1,2,55369,2,1,0
597,0,2,154058,0,1,0
598,1,1,180083,17,0,0


In [15]:
a=data2.iloc[:,0:5]
b=data2['Tax_Cat']
a 

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban
0,0,0,50047,10,1
1,1,2,134075,18,1
2,0,1,160205,30,1
3,1,0,193264,15,1
4,0,1,27533,28,0
...,...,...,...,...,...
595,1,2,39492,7,1
596,1,2,55369,2,1
597,0,2,154058,0,1
598,1,1,180083,17,0


In [16]:
a_train,a_test,b_train,b_test =train_test_split(a,b,test_size =0.3,random_state =42)

In [18]:
model =RandomForestClassifier(n_estimators=100,max_depth =3, criterion='gini',random_state=42)
model.fit(a_train,b_train)
b_pred =model.predict(a_test)
b_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [19]:
accuracy =accuracy_score(b_test,b_pred)
print(accuracy) 

0.7944444444444444


In [23]:
print(classification_report(b_test,b_pred))

              precision    recall  f1-score   support

           0       0.79      1.00      0.89       143
           1       0.00      0.00      0.00        37

    accuracy                           0.79       180
   macro avg       0.40      0.50      0.44       180
weighted avg       0.63      0.79      0.70       180



/Users/uttejkumar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/uttejkumar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/uttejkumar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

In [28]:
num_trees = 100
kfold = KFold(n_splits=10, shuffle = True, random_state=42)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=42)
results = cross_val_score(model, a,b, cv=kfold)
print(results.mean())

0.775


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [33]:
kfold = KFold(n_splits=10,shuffle=True, random_state=42)
estimators = []
model1 = LogisticRegression(max_iter=100)                          
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))

In [35]:
from sklearn.ensemble import VotingClassifier

In [37]:
ensemble = VotingClassifier(estimators)                            
results = cross_val_score(ensemble, a, b, cv=kfold)
print(results.mean()) 

0.7949999999999999
